In [1]:
#Python Library
import warnings
warnings.filterwarnings("ignore")
import math
import numpy as np
import pandas as pd
import pickle

# ML Library

In [2]:
#ML Library
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, confusion_matrix, classification_report

from sklearn.model_selection import train_test_split

# Load Dataset

In [3]:
#load data
second_innings_win_portability_df = pickle.load(open('psl_mens_second_innings_win.pkl','rb'))
second_innings_win_portability_df

,batting_team,bowling_team,city,current_score,target_runs,runs_left,balls_left,wickets_left,crr,rrr,result
0,Islamabad United,Peshawar Zalmi,Dubai,4,191,187,119,10,24.00,9.43,1
1,Islamabad United,Peshawar Zalmi,Dubai,4,191,187,118,10,12.00,9.51,1
2,Islamabad United,Peshawar Zalmi,Dubai,8,191,183,117,10,16.00,9.38,1
3,Islamabad United,Peshawar Zalmi,Dubai,9,191,182,116,10,13.50,9.41,1
4,Islamabad United,Peshawar Zalmi,Dubai,10,191,181,115,10,12.00,9.44,1
...,...,...,...,...,...,...,...,...,...,...,...
27074,Islamabad United,Quetta Gladiators,Dubai,169,175,6,12,7,9.39,3.00,1
27075,Islamabad United,Quetta Gladiators,Dubai,173,175,2,11,7,9.52,1.09,1
27076,Islamabad United,Quetta Gladiators,Dubai,173,175,2,10,6,9.44,1.20,1
27077,Islamabad United,Quetta Gladiators,Dubai,174,175,1,9,6,9.41,0.67,1


# Test and Train Split

In [4]:
#Split
X2 = second_innings_win_portability_df.drop(columns=['result'])
y2 = second_innings_win_portability_df['result'].values

X2_train,X2_test,y2_train,y2_test = train_test_split(X2, y2, test_size=0.1, random_state=1)

In [5]:
#Display
X2_train

,batting_team,bowling_team,city,current_score,target_runs,runs_left,balls_left,wickets_left,crr,rrr
2563,Quetta Gladiators,Peshawar Zalmi,Lahore,36,149,113,79,6,5.27,8.58
25313,Quetta Gladiators,Lahore Qalandars,Dubai,43,195,152,83,7,6.97,10.99
12215,Peshawar Zalmi,Multan Sultans,Karachi,14,155,141,100,8,4.20,8.46
8304,Karachi Kings,Lahore Qalandars,Dubai,27,134,107,97,9,7.04,6.62
703,Islamabad United,Quetta Gladiators,Sharjah,18,149,131,99,10,5.14,7.94
...,...,...,...,...,...,...,...,...,...,...
10985,Karachi Kings,Multan Sultans,Multan,69,187,118,67,6,7.81,10.57
17333,Lahore Qalandars,Karachi Kings,Karachi,76,171,95,62,8,7.86,9.19
5208,Quetta Gladiators,Lahore Qalandars,Sharjah,108,187,79,42,5,8.31,11.29
12204,Peshawar Zalmi,Multan Sultans,Karachi,3,155,152,111,9,2.00,8.22


In [6]:
#Column Transformation
trf2 = ColumnTransformer([
    ('trf2', OneHotEncoder(sparse=False, drop='first'),['batting_team', 'bowling_team', 'city'])
]
,remainder='passthrough')

In [7]:
#Pipeline
pipe2 = Pipeline(steps=[
    ('step1',trf2),
    ('step2',LogisticRegression())
])

In [8]:
#Train Model
pipe2.fit(X2_train,y2_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf2',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression())])

In [9]:
pipe2.predict_proba(X2_test)[10]

array([0.1673369, 0.8326631])

In [10]:
#Accuracy on test data
y2_pred = pipe2.predict(X2_test)

In [11]:
#Confusion Matrix
confusion_matrix(y2_test, y2_pred)

array([[ 952,  249],
       [ 240, 1258]])

In [12]:
#Classiffication Report
print(classification_report(y2_test, y2_pred))

              precision    recall  f1-score   support

           0       0.80      0.79      0.80      1201
           1       0.83      0.84      0.84      1498

    accuracy                           0.82      2699
   macro avg       0.82      0.82      0.82      2699
weighted avg       0.82      0.82      0.82      2699



# Result Prediction

In [28]:
#Prediction
input_df = pd.DataFrame({'batting_team': ['Islamabad United'], 'bowling_team': ['Peshawar Zalmi'], 'city': ['Dubai'], 'current_score':[100], 'target_runs':[169], 'runs_left':[100], 'balls_left': [69], 'wickets_left':[7], 'crr':[8.11], 'rrr': [8.69]})
result = pipe2.predict_proba(input_df)

loss = result[0][0]
win = result[0][1]

print('win: '+str(round(win*100))+'%')
print('loss: '+str(round(loss*100))+'%')

win: 75%
loss: 25%


# Model Extraction

In [29]:
#Model Extraction
pickle.dump(pipe2, open('psl_mens_second_innings_final_model.pkl','wb'))